In [ ]:
import datetime
from typing import Union, Tuple
from matplotlib import pyplot as plt
from IPython.display import display, Markdown as md

from systematic.strategies import helper as strategies
from notebooks.params import REF_DATE, BT_START_DATE, WEIGHTS, DISTANCE, SSL_STRATS, SHARPE_CHART_CAP, SHARPE_CHART_FLOOR

In [ ]:
# Settings for export
# *** can safely skip this Cell when running in interactive mode ***

import warnings
import sylo.com.progress_bar

# render chart as image
%matplotlib inline
warnings.filterwarnings('ignore')  # ignore warning messages
sylo.com.progress_bar.GLOBAL_PROGRESS_BAR_ON = False  # switch off progress bar

In [ ]:
# REF_DATE = datetime.date(2021, 1, 1)
# BT_START_DATE = datetime.date(2010, 1, 1)
print(f'REF_DATE set to {REF_DATE}')
REPORT_DATETIME = datetime.datetime.now()
print(f'REPORT_DATETIME is {REPORT_DATETIME}')
allocation = strategies.get_allocation()

## Portfolio

In [ ]:
def get_and_plot(strat: Union[str, Tuple[str]]):
    data1 = strategies.get_minmax_envelope(strat, start_date=REF_DATE, envelope_periods=252, backtest_allocation=WEIGHTS, backtest_start_date= BT_START_DATE)
    data2 = strategies.get_minmax_envelope(strat, start_date=REF_DATE, envelope_periods=252, backtest_allocation=WEIGHTS, backtest_start_date= BT_START_DATE, backtest_condition_value_range=DISTANCE)
    data2 = data2.loc[datetime.date.today():datetime.date(datetime.date.today().year,12,31)]
    data3 = strategies.get_sharpe_envelope(strat, start_date=REF_DATE, envelope_periods=252, backtest_allocation=WEIGHTS, backtest_start_date= BT_START_DATE)
    data3.clip(lower=SHARPE_CHART_FLOOR, upper=SHARPE_CHART_CAP, inplace=True)
    fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(12, 5))
    fig.tight_layout()
    _ = data1.plot(title=SSL_STRATS.get(strat), color='rgybcm', ax=ax1)
    ax1.grid(axis='y', linestyle='--')
    _ = data2.plot(title=SSL_STRATS.get(strat), color='rgybcm', ax=ax2)
    ax2.grid(axis='y', linestyle='--')
    _ = data3.plot(title=SSL_STRATS.get(strat), color='rgybcm', ax=ax3)
    ax3.grid(axis='y', linestyle='--')
    plt.show()

In [ ]:
PORTFOLIO = tuple(WEIGHTS)
get_and_plot(PORTFOLIO)

In [ ]:
REF_END_DATE = datetime.date(REF_DATE.year, 12, 31)
display(strategies.get_bt_hist(PORTFOLIO, start_date=REF_DATE, end_date=REF_END_DATE, backtest_allocation=WEIGHTS, backtest_start_date=BT_START_DATE, backtest_condition_value_range=DISTANCE))

## Individual Strategies

In [ ]:
for strat in WEIGHTS:
    get_and_plot(strat)

In [ ]:
display(md(f'### Projected P&L Distribution on {REF_END_DATE}'))

In [ ]:
data5 = strategies.get_bt_hist(PORTFOLIO, start_date=REF_DATE, end_date=REF_END_DATE, backtest_allocation=WEIGHTS, backtest_start_date=BT_START_DATE, backtest_condition_value_range=DISTANCE, return_raw=True)
_ = data5.hist(figsize=(12, 12))